<a href="https://colab.research.google.com/github/sunny-5555/Computer-Vision-Assignments/blob/add-notebooks/a2-convolution-panoramaStitching/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Vision - Assignment 2: Convolution, Cartoonization and Panorama Stitching

---

Prof. Dr. Markus Enzweiler, Esslingen University of Applied Sciences

markus.enzweiler@hs-esslingen.de

---

This is the second assignment for the "Machine Vision" lecture. 
It covers:
* implementing your own image convolution function
* applying OpenCV functions for Canny edge detection, bilateratal filtering, cartoonization
* implementing an image panorama stitching function based on SIFT and RANSAC

To successfully complete this assignment, it is assumed that you already have some experience in Python and numpy. You can either use [Google Colab](https://colab.research.google.com/) for free with a private (dedicated) Google account (recommended) or a local Jupyter installation.

---


## Preparations


### Import important libraries (you should probably start with these lines all the time ...)

In [ ]:
# OpenCV
import cv2   

# NumPy                    
import numpy as np     

# Matplotlib    
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# make sure we show all plots directly below each cell
%matplotlib inline 

# Request
import requests

# Some Colab specific packages
if 'google.colab' in str(get_ipython()):
  # image display
  from google.colab.patches import cv2_imshow 


### Some helper functions that we will need

In [ ]:
def my_imshow(image, windowTitle="Image"):
  '''
  Displays an image and differentiates between Google Colab and a local Python installation. 

  Args: 
    image: The image to be displayed

  Returns:
    - 
  '''

  if 'google.colab' in str(get_ipython()):
    cv2_imshow(image)
  else:
    cv2.imshow(windowTitle, image)

## Exercise 1 - Image Convolution (10 points) 

*Note: This is approx. 20-25 lines of Python code to implement.*

Implement a convolution function ```myConvolution(inImage, kernel)``` that takes in as input an arbitrary image and convolution kernel. Use only your own code, i.e. no OpenCV or other external libraries providing a dedicated convolution function. At the boundaries of the image, assume that the values outside the image are zero. A good strategy to do so is to pad the image with zeros prior to convolution  Padding involves adding additional rows and columns containing zeros at the image boundaries and removing the padding after convolution. 

Test your function with the provided image and kernel. Your output should look identical (maybe minor differences at the image boundaries due to different border handling) to the OpenCV reference ```cv2.filter2D()```that is provided. How does your implementation compare to the OpenCV implementation regarding computation time? You can use ```%%time``` for timing measurements. 

---

The following code contains a code template as well as an OpenCV implementation of image convolution as reference. Please add your own implementation to the function ```myConvolution(inImage, kernel)```. 

---

### Load and display the input image

In [ ]:
# Load and display the input image 

url = 'https://raw.githubusercontent.com/sunny-5555/Computer-Vision-Assignments/main/a2-convolution-panoramaStitching/convolution/esslingen800px.jpg'
response = requests.get(url, allow_redirects = True, stream = True).raw
image = np.asarray(bytearray(response.read()), dtype="uint8")

# read image and convert to gray
inputImage = cv2.imdecode(image, cv2.IMREAD_COLOR)
inputImage = cv2.cvtColor(inputImage, cv2.COLOR_BGRA2GRAY)

# Display the input image
print("Input image")
my_imshow(inputImage)

### Create a Gaussian filter kernel

In [ ]:
# Create a Gaussian filter kernel
kernel = 1.0/16.0 * np.array([[1,2,1], [2,4,2], [1,2,1]])
print("Gaussian Kernel\n{}".format(kernel))

### Apply the filter using OpenCV's ```filter2D()``` function. 

In [ ]:
%%time
outputImage = cv2.filter2D(inputImage, -1, kernel, borderType=cv2.BORDER_CONSTANT)
print("OpenCV filter2D() convolution result")
my_imshow(outputImage)
print(outputImage)

### Implement your own convolution function ```myConvolution()``` (**add your code here**)


In [ ]:
# Template for your own convolution function

def myConvolution(inImage, kernel):
  '''
  2D image convolution of an image with a filter kernel.

  Args:   
    inImage: input image
    kernel: input 2D filter kernel

  Returns:
    outImage: output image, the input image convolved with the filter kernel
  '''

  padding_border_width = 1
  padding_value = 0
  inImagePad = np.pad(inImage, padding_border_width, 'constant', constant_values=padding_value)

  avg_win_size = 3
  outImage = np.copy(inImage)

  for x in range(0,len(inImage)):
    for y in range(0,len(inImage[x])):
      averaging_window = np.empty((0,avg_win_size), int)
      for x_avg in range(0,avg_win_size):
        coppied_arr = np.copy(inImagePad[x+x_avg][y:y+avg_win_size])
        averaging_window = np.append(averaging_window, [coppied_arr],axis=0)
      products = np.multiply(kernel, averaging_window)
      convoluted_el = products.sum()
      outImage[x,y] = convoluted_el

  return outImage
outputImage = myConvolution(inputImage, kernel)

### Apply and time your own convolution function 


In [ ]:
# Run your own convolution function
%%time
myoutputImage = myConvolution(inputImage, kernel)
print("myConvolution() convolution result")
my_imshow(outputImage)

In [ ]:
rows = len(outputImage)
for row in range(0,rows-1):
  for n in range(0,len(outputImage[row])-1):
    if outputImage[row][n] != myoutputImage[row][n]:
      print(f"Element {row},{n} not equal\noutputImage[row][n] = {outputImage[row][n]}\nmyoutputImage[row][n] = {myoutputImage[row][n]}")

## Exercise 2 - Canny Edge Detection and Cartoonization with OpenCV (10 points)

*Note: This is approx. 10 lines of Python code to implement.*

1.   Use your favorite search engine to find out how to do run the Canny Edge Detector in OpenCV, as we have seen in the lecture. Use it to detect edges in the provided image ```ironman.jpg``` and visualize the result. 

2.   Use your favorite search engine to find out how to do to cartoonization in OpenCV as a combination of edge detection and bilateral filtering, as we have seen in the lecture. Apply it to the image ```ironman.jpg``` and visualize the result. 

Your results should look similar to the following images (see ```result.jpg```). You can right-click on this image and "open in new tab/window" to display it in its original size:

<img src='https://drive.google.com/uc?id=16Zb5fsUfaGWi0YI39pezJ8vu-3UlWED4'></img>


In [ ]:
# get image
url = 'https://raw.githubusercontent.com/sunny-5555/Computer-Vision-Assignments/main/a2-convolution-panoramaStitching/canny_cartoonization/ironman.jpg'
response = requests.get(url, allow_redirects = True, stream = True).raw
image = np.asarray(bytearray(response.read()), dtype="uint8")

# read image
inputImage = cv2.imdecode(image, cv2.IMREAD_COLOR)

# Display the input image
print("Input image")
my_imshow(inputImage)

### Canny Edge Detection

In [ ]:
cannyImage = cv2.Canny(inputImage,110,200)
print("Canny edge detection image")
cv2_imshow(cannyImage)
print(cannyImage)
print(cannyImage.shape)
print(inputImage.shape)

### Cartoonization

In [ ]:
inImage_bil_filter = cv2.bilateralFilter(inputImage,15,80,80) #bilateral_blur = cv2.bilateralFilter(img,15,80,80)  9,75,75
cannyImage = cv2.Canny(inImage_bil_filter,110,200)
print("Canny edge detection image")
cv2_imshow(cannyImage)
print(cannyImage)
cartoon = cv2.subtract(inImage_bil_filter, cv2.cvtColor(cannyImage, cv2.COLOR_GRAY2BGR))
my_imshow(cartoon)

## Exercise 3 - Panorama Stitching with SIFT and RANSAC (10 points)

*Note: This is approx. 30-40 lines of Python code to implement.*

In this exercise you will work on image panorama stitching using SIFT and RANSAC. In principle, this exercise follows the exact pipeline that we have discussed in the lecture:


1.   Compute SIFT feature descriptors for both input images (**provided, not to be implemented**).
2.   Compute distances between every SIFT descriptor in one image and every descriptor in the other image (**provided, not to be implemented**).
3. Select closest matches based on the matrix of pairwise descriptor distances obtained above. We will select the top 2500 descriptor pairs with the smallest pairwise distances (**provided, not to be implemented**).
4. Implement a RANSAC-based estimation of the homography between the two provided images (***to be implemented***) that is can be used to stitch the two images into a single panoramic image. An OpenCV reference implementation is provided for you to compare against. 

### Preparations and provided functions



#### Packages, Imports and Drive Mounting

In [ ]:
# We will need to install a different opencv package and restart the Colab runtime, see below. 
# All the imports are reproduced here again so that you do not need to run anything from the 
# previous two exercises after restarting the Colab runtime. 


# Imports 

import sys
import random

# OpenCV
import cv2   

# NumPy                    
import numpy as np     

# Matplotlib    
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# make sure we show all plots directly below each cell
%matplotlib inline 

# Some Colab specific packages
if 'google.colab' in str(get_ipython()):
  # image display
  from google.colab.patches import cv2_imshow 



# For SIFT, we need to install a more recent opencv version that what is 
# provided in the Colab runtime :(
    
OpenCVVersion = "4.5.1.48" # which version shall be installed?

if 'google.colab' in sys.modules and (cv2.__version__ not in OpenCVVersion) :
    import subprocess
    print("Installing opencv-contrib-python version {}".format(OpenCVVersion))
    !pip uninstall opencv-python -y
    !pip install opencv-contrib-python==$OpenCVVersion
    print('Please manually restart Colab runtime ...')
   
print("Installed OpenCV version is {}".format(cv2.__version__))



# Mount Google Drive
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)


#### Provided Functions

In [ ]:
# Display an image
def my_imshow(image, windowTitle="Image"):
  '''
  Displays an image and differentiates between Google Colab and a local Python installation. 

  Args: 
    image: The image to be displayed

  Returns:
    - 
  '''

  if 'google.colab' in str(get_ipython()):
    cv2_imshow(image)
  else:
    cv2.imshow(windowTitle, image)



# Provided function to compute an homography between two sets of points p1 and p2 based on least squares
# Input format is the same as for cv2.findHomography()
def myComputeHomography(p1, p2):
  '''
  Estimates an homography between two sets of points p1 and p2 based on least squares.  
  Input format is the same as for cv2.findHomography(). At least 4 input points are needed in each input set. 
  '''
  A = []
  for i in range(0, len(p1)):
      x, y = p1[i][0][0], p1[i][0][1]
      u, v = p2[i][0][0], p2[i][0][1]
      A.append([x, y, 1, 0, 0, 0, -u*x, -u*y, -u])
      A.append([0, 0, 0, x, y, 1, -v*x, -v*y, -v])
  A = np.asarray(A)
  U, S, Vh = np.linalg.svd(A)
  L = Vh[-1,:] / Vh[-1,-1]
  return L.reshape(3, 3)



# Provided function to stitch two images together using the provided homography
def myStitchImages(imageLeft, imageRight, homography):
  '''
  Stitch two images together using the provided homography
  '''
  # Reserve some space for the final stitched image
  stitchedImageHeight = imageLeft.shape[0] + imageRight.shape[0]
  stitchedImageWidth  = imageLeft.shape[1] + imageRight.shape[1]

  # Warp the right image 
  stitchedImage = cv2.warpPerspective(imageRight, homography, dsize = (stitchedImageWidth, stitchedImageHeight))

  # Copy the left image into the stitched image
  stitchedImage[0:imageLeft.shape[0], 0:imageLeft.shape[1], :] = imageLeft

  return stitchedImage


### Compute and visualize SIFT keypoints and descriptors for both input images (**provided**) 

In [ ]:
# Load the input images and convert to gray

url_base = 'https://raw.githubusercontent.com/sunny-5555/Computer-Vision-Assignments/main/a2-convolution-panoramaStitching/panoramaStitching/'

response = requests.get(url_base + "esslingenPanoLeft.jpg", allow_redirects = True, stream = True).raw
image = np.asarray(bytearray(response.read()), dtype="uint8")
inputImageLeftColor = cv2.imdecode(image, cv2.IMREAD_COLOR)
inputImageLeft      = cv2.cvtColor(inputImageLeftColor, cv2.COLOR_BGR2GRAY)

response = requests.get(url_base + "esslingenPanoRight.jpg", allow_redirects = True, stream = True).raw
image = np.asarray(bytearray(response.read()), dtype="uint8")
inputImageRightColor = cv2.imdecode(image, cv2.IMREAD_COLOR)
inputImageRight      = cv2.cvtColor(inputImageRightColor, cv2.COLOR_BGR2GRAY)



# Following should be uncommented if the images on Google Drive shall be used
# imPath = "/content/drive/My Drive/Machine Vision/MV_assignments/a2-convolution-panoramaStitching/panoramaStitching/"

# inputImageLeftColor = cv2.imread(imPath + "esslingenPanoLeft.jpg")
# inputImageLeft      = cv2.cvtColor(inputImageLeftColor, cv2.COLOR_BGR2GRAY)

# inputImageRightColor = cv2.imread(imPath + "esslingenPanoRight.jpg")
# inputImageRight      = cv2.cvtColor(inputImageRightColor, cv2.COLOR_BGR2GRAY)

# Display the input images
print("Input image (left)")
my_imshow(inputImageLeftColor)

print("Input image (right)")
my_imshow(inputImageRightColor)

In [ ]:
# Compute SIFT features for both the left and right image

# Left image
# Apply SIFT feature detector 
sift = cv2.SIFT_create()
keyPointsLeft, descriptorsLeft = sift.detectAndCompute(inputImageLeft,None)
siftImageLeft = cv2.drawKeypoints(inputImageLeft, keyPointsLeft, inputImageLeft, \
                                  flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display image
print("Found {} {}-dimensional SIFT Features (left)".format(descriptorsLeft.shape[0], descriptorsLeft.shape[1]))
my_imshow(siftImageLeft)

# Right image
# Apply SIFT feature detector 
sift = cv2.SIFT_create()
keyPointsRight, descriptorsRight = sift.detectAndCompute(inputImageRight,None)
siftImageRight = cv2.drawKeypoints(inputImageRight, keyPointsRight, inputImageRight, \
                                   flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display image
print("Found {} {}-dimensional SIFT Features (right)".format(descriptorsRight.shape[0], descriptorsRight.shape[1]))
my_imshow(siftImageRight)


### Compute distances between every SIFT descriptor in one image and every descriptor in the other image. Select the top 2500 matches (**provided**).

In [ ]:
# Match SIFT descriptors between two images
# Brute force matching: For every SIFT descriptor in the left image, the best matching 
# (= most similar SIFT descriptor) in the right image is found.
# See: OpenCV BFMatcher class reference

matcher = cv2.BFMatcher(normType = cv2.NORM_L2)
matches = matcher.match(descriptorsLeft, descriptorsRight) 
print("# of matches: {}".format(len(matches)))


# Only take the best "n" matches, i.e. the ones with the smallest distance between SIFT descriptor vectors

numMatchesUsed = 2500
# Sort them in the order of their distance and only take "n"
matches = sorted(matches, key = lambda x:x.distance)[:numMatchesUsed]

# Extract the locations of matched keypoints in both images
locationsLeft  = np.float32( [ keyPointsLeft  [match.queryIdx].pt for match in matches]).reshape(-1,1,2)
locationsRight = np.float32( [ keyPointsRight [match.trainIdx].pt for match in matches]).reshape(-1,1,2)

print("# of selected matches: {}".format(len(locationsLeft)))

### OpenCV reference implementation of RANSAC-based homography estimation and stitching (**provided for reference**)

In [ ]:
# RANSAC parameters (for both OpenCV and your implementation)

rNumTrials = 250
rMatchThresh = 7.0 # in pixels

# OpenCV computed homography for reference
openCVHomography, mask = cv2.findHomography(locationsRight, locationsLeft, cv2.RANSAC, \
                             ransacReprojThreshold = rMatchThresh, maxIters = rNumTrials)
print("OpenCV computed homography (for reference) is: \n{}". format(openCVHomography))


# Stitch
stitchedImage = myStitchImages(inputImageLeftColor, inputImageRightColor, openCVHomography)

print("Result using OpenCV reference implementation")
my_imshow(stitchedImage)

### Your own RANSAC-based homography estimation and stitching (**to be implemented**)

In [ ]:
# RANSAC parameters
rNumTrials = 250
rMatchThresh = 7.0 # in pixels


# variables
inlierFraction = 0
rBestInlierFraction = 0 
rBestTrial = 0
num_inliers = 0

# a dummy homography
rBestHomography =  np.identity(3)
rBestHomography[0,2] = 1500

# Main RANSAC loop
print("Starting RANSAC loop with {} iterations".format(rNumTrials))

for iTrial in range(1, rNumTrials):
  # 1) Select 4 random matches out of the 2500. The actual point coordinates are in the "keyPointsLeft"
  #    and "keyPointsRight" array. See how the 2500 matches were selected out of the full set. Now, we need
  #    4 out of 2500 as 4 is the minimum number of points that is needed to compute an homography.
  
  
  idxes = np.random.randint(0, numMatchesUsed-1, 4)
  randomMatchesLeft = []
  randomMatchesRight = []
  for idx in idxes:
    randomMatchesLeft.append(locationsLeft[idx])
    randomMatchesRight.append(locationsRight[idx])

  # 2) Compute the homography using the provided myComputeHomography() function, see above. 
  
  homographieRandomMatches = myComputeHomography(randomMatchesRight,randomMatchesLeft)

  # 3) Evaluate quality of the homography by computing the fraction of inliers through all 2500 matches. 
  #    Inliers are defined as the number of warped points from the right image (using the homography) that lie
  #    within a user-defined distance "rMatchThresh" pixels of their corresponding point in image left image as 
  #    defined by the SIFT matching. 

  num_inliers = 0  
  # for all 2500 matches
  for iMatch in range(0, len(matches)):    
    ;
    # Get the coordinates of left and right points for the current "match"
    # (in homogeneous coordinates, i.e. a 3-dimensional vector with an added "1.0" as third element)
  
    leftPt = locationsLeft[iMatch]
    left3dVector = np.array([leftPt[0][0],leftPt[0][1],1.0]).reshape((1,3))
    rightPt = locationsRight[iMatch]
    right3dVector = np.array([rightPt[0][0],rightPt[0][1],1.0]).reshape((1,3))
    # Warp the right point to the left image using the homography. 
    # Convert back from homogenous coordinates by dividing the first and second element of the vector by the third
   
    warpRight = np.matmul(homographieRandomMatches,right3dVector.T)
    right2dWarp = np.array([warpRight[0],warpRight[1]])
    right2dWarp = 1/warpRight[2] * right2dWarp

    # Compute euclidean distance between the original left point and the right point warped to the left image
   
    dist = np.linalg.norm(leftPt-right2dWarp.T)

    # Inlier or outlier based on the euclidean distance and rMatchThresh
    
    if dist < rMatchThresh:
      num_inliers += 1

  # --------- loop over all matches ends here --------------


  # 4) Use the fraction of inliers (num inliers / all matches) to evaluate the estimated homography. 
  #    If it is better than a previous one, store it. 
    
  # 5) Some status output

  inlierFraction = num_inliers / len(matches)

  if rBestInlierFraction < inlierFraction:
    rBestInlierFraction = inlierFraction
    rBestHomography = homographieRandomMatches
    rBestTrial = iTrial

  print("---------------------------------------------------------------------------------------") 
  print("RANSAC iteration [{}] : Found transform with {:0.2f}% inliers".format(iTrial, inlierFraction * 100))
  print("Current best transform is in iteration {} with {:0.2f}% inliers".format(rBestTrial, rBestInlierFraction * 100))

# --------- end of a single RANSAC iteration --------------



# 6) RANSAC loop (all iterations) completed

print("---------------------------------------------------------------------------------------") 
print("RANSAC loop completed")
print("Best transform is in iteration {} with {:0.2f}% inliers".format(rBestTrial, rBestInlierFraction * 100))
print("Estimated Homography: \n{}".format(rBestHomography))

# Stitch using the best estimated homography
stitchedImage = myStitchImages(inputImageLeftColor, inputImageRightColor, rBestHomography)

print("Result using implemented RANSAC")
my_imshow(stitchedImage)